In [414]:
import pandas as pd
import numpy as np

In [415]:
formerCast_df = pd.read_csv('formerCast.csv')
currentCast_df = pd.read_csv('currentCast.csv')
specialCast_df = pd.read_csv('specialCast.csv')

In [416]:
fc_episode = pd.read_csv('fc_episodes.csv')
sc_episode = pd.read_csv('sc_episodes.csv')
cc_episode = pd.read_csv('cc_episodes.csv')

In [417]:
fc_episode = fc_episode[fc_episode['season'] > 0]
sc_episode = sc_episode[sc_episode['season'] > 0]
cc_episode = cc_episode[cc_episode['season'] > 0]

In [418]:
fc_episode = fc_episode.reset_index(drop=True)
sc_episode = sc_episode.reset_index(drop=True)
cc_episode = cc_episode.reset_index(drop=True)

In [419]:
fc_episode['startEp'] = fc_episode['startEp'].astype('int')
fc_episode['endEp'] = fc_episode['endEp'].astype('int')
fc_episode = fc_episode.set_index('father')

sc_episode['startEp'] = sc_episode['startEp'].astype('int')
sc_episode['endEp'] = sc_episode['endEp'].astype('int')
sc_episode = sc_episode.set_index('father')

cc_episode['startEp'] = cc_episode['startEp'].astype('int')
cc_episode['endEp'] = cc_episode['endEp'].astype('int')
cc_episode = cc_episode.set_index('father')

In [420]:
def encoding_episodes(row):
    row.loc[row['startEp']-1 + (4): row['endEp']+ (4)] = str(row['dad'])
    return row

def get_table(episode_df):
    #start_df = pd.get_dummies(episode_df['startEp'])
    start_df = episode_df.copy()
    i = 1
    while i <= 304:
        start_df[i] = None
        i = i + 1
    #start_df['totalNoOfEp'] = fc_episode['totalNoOfEp']
    #
    start_df['dad'] = episode_df.index
    
    
    start_df = start_df.apply(encoding_episodes, axis=1)
    #start_df = pd.pivot_table(start_df, values=intList, columns=['father'], aggfunc='max')
    start_df = pd.pivot_table(start_df, values=intList, columns=['father'], aggfunc = lambda x: x.any() if x.notnull().values.any() else None)
    return start_df

In [421]:
fc_encoded_df = get_table(fc_episode)
sc_encoded_df = get_table(sc_episode)
cc_encoded_df = get_table(cc_episode)

In [281]:
'''
sampleDF = pd.DataFrame({'dad': ['Dad A', 'Dad B', 'Dad C'],
                        '1': [1, 0, 0],
                        '2': [1, 1, 0],
                        '3': [0, 0, 1]})
'''

In [282]:
#pd.pivot_table(sampleDF, values=['1', '2', '3'], columns=['dad'])

dad,Dad A,Dad B,Dad C
1,1,0,0
2,1,1,0
3,0,0,1


In [325]:
#x = sampleDF.stack()
#x.index = x.index.droplevel()
#x
#x.drop(['dad'], axis='rows')

In [272]:
#fc_encoded_df['episode'] = fc_encoded_df.index
#sc_encoded_df['episode'] = sc_encoded_df.index
#cc_encoded_df['episode'] = cc_encoded_df.index

In [446]:
fc_encoded_df = pd.DataFrame(fc_encoded_df.stack())
fc_encoded_df.columns = ['father']
fc_encoded_df.index = fc_encoded_df.index.droplevel(1)

sc_encoded_df = pd.DataFrame(sc_encoded_df.stack())
sc_encoded_df.columns = ['father']
sc_encoded_df.index = sc_encoded_df.index.droplevel(1)

cc_encoded_df = pd.DataFrame(cc_encoded_df.stack())
cc_encoded_df.columns = ['father']
cc_encoded_df.index = cc_encoded_df.index.droplevel(1)

In [451]:
total_encoded_df = pd.concat([fc_encoded_df,sc_encoded_df,cc_encoded_df])

In [447]:
import os
directory = 'data'
if not os.path.exists(directory):
    os.makedirs(directory)

In [452]:
dict_df = {'fc_encoded.csv': fc_encoded_df, 
           'sc_encoded.csv': sc_encoded_df, 
           'cc_encoded.csv': cc_encoded_df,
            'total_encoded.csv': total_encoded_df}

for key,val in dict_df.items():
    path = os.path.join(directory, key)
    val.to_csv(path, encoding='utf-8')

KeyError: "['episode'] not found in axis"

In [405]:
fc_encoded_df = pd.DataFrame(fc_encoded_df.stack())
fc_encoded_df.columns = ['father']
fc_encoded_df.index = fc_encoded_df.index.droplevel(1)

In [442]:
cc_encoded_df

father,Hong Kyung-min,Moon Hee-joon,Park Joo-ho,Sam Hammington
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,None,None,None,None
8,None,None,None,None
9,None,None,None,None
10,None,None,None,None


In [438]:
x3 = cc_encoded_df.stack()

In [439]:
x3

     father        
154  Sam Hammington    Sam Hammington
155  Sam Hammington    Sam Hammington
156  Sam Hammington    Sam Hammington
157  Sam Hammington    Sam Hammington
158  Sam Hammington    Sam Hammington
159  Sam Hammington    Sam Hammington
160  Sam Hammington    Sam Hammington
161  Sam Hammington    Sam Hammington
162  Sam Hammington    Sam Hammington
163  Sam Hammington    Sam Hammington
164  Sam Hammington    Sam Hammington
165  Sam Hammington    Sam Hammington
166  Sam Hammington    Sam Hammington
167  Sam Hammington    Sam Hammington
168  Sam Hammington    Sam Hammington
169  Sam Hammington    Sam Hammington
170  Sam Hammington    Sam Hammington
171  Sam Hammington    Sam Hammington
172  Sam Hammington    Sam Hammington
173  Sam Hammington    Sam Hammington
174  Sam Hammington    Sam Hammington
175  Sam Hammington    Sam Hammington
176  Sam Hammington    Sam Hammington
177  Sam Hammington    Sam Hammington
178  Sam Hammington    Sam Hammington
179  Sam Hammington    Sam Ham

In [440]:
x3 = pd.DataFrame(x3)

In [441]:
x3

,,0
,father,
154,Sam Hammington,Sam Hammington
155,Sam Hammington,Sam Hammington
156,Sam Hammington,Sam Hammington
157,Sam Hammington,Sam Hammington
158,Sam Hammington,Sam Hammington
159,Sam Hammington,Sam Hammington
160,Sam Hammington,Sam Hammington
161,Sam Hammington,Sam Hammington
162,Sam Hammington,Sam Hammington


In [443]:
x3.columns = ['father']

In [444]:
x3.index = x3.index.droplevel(1)

In [445]:
x3

,father
154,Sam Hammington
155,Sam Hammington
156,Sam Hammington
157,Sam Hammington
158,Sam Hammington
159,Sam Hammington
160,Sam Hammington
161,Sam Hammington
162,Sam Hammington
163,Sam Hammington


In [379]:
pd.DataFrame(x2).to_csv("WATTH.csv", encoding='utf-8')

In [373]:
#x2 = x2.drop(['episode'], axis='rows')

KeyError: "['episode'] not found in axis"

In [375]:
pd.DataFrame(x2).to_csv("WATTH.csv", encoding='utf-8')